In [25]:
#Imports
import numpy as np
import sympy as sym
import h5py as h5
from matplotlib import pyplot as plt
from scipy.integrate import quad
from scipy.integrate import dblquad
from scipy.integrate import nquad
import scipy.optimize as so
import scipy.special as ss
from scipy.special import ellipe
from scipy.special import ellipk

#Settings
options={'limit':100} #Set maximum number of subdivisions to 100 instead of 50.

In [26]:
#Constants
Mbh = 2.7e9                                                 #mass of the central black hole in (solar mass)
G = 4.300e-6                                                #gravitational constant (kpc/solar mass*(km/s)^2)
h = 8.9                                                     #radial scale-length (kpc)
rho00 = 0.31e9                                              #prefactor that will cancel
epsdisk = 5.0                                               #from Noordermeer's paper
absmag = -22.02                                             #absolute magnitude 
magsun = 4.42                                               #absolute magnitude of the sun
z0 = 0.2*h                                                  #half-thickness (kpc)
R = 4*h                                                     #cut-off radius (kpc)
d = 0.2*h                                                   #cut-off length upper limits (kpc)
L0 = np.power(10, (0.4*(magsun-absmag)))                    #Absolute Magnitude to luminosity
n = 2.7                                                     #concentration parameter that describes the curvature of the profile in a radius-magnitude plot, n=4 is de Vaucoileurs profile
re = 2.6                                                    #1kpc
ups = 2.8                                                   #mass-to-light ratio (from Rotation Curves of Sersic Bulges paper)
q = 0.33                                                    #intrinsic axis ratio
i = 45*(np.pi/180)                                          #inclination angle
L = 3.27e10                                                 #luminosity

In [27]:
#Variables
x = np.linspace(1, 10, 100)                        #x from/to and line smoothness

In [34]:
#Run necessary code using existing imports and variables

%run -i Scripts/bulge.py                    #Time: Roughly 2 minutes if building, Almost 0 if not.
%run -it Scripts/disk.py                    #Time: 40 Minutes if building,
%run -i Scripts/halo.py                     #Time: Almost 0
%run -i Scripts/blackhole.py                #Time: Almost 0

Scripts/disk.py:51: RuntimeWarning: overflow encountered in cosh
  #Partial Derivative of rho(u,xi)
Scripts/disk.py:53: RuntimeWarning: invalid value encountered in double_scalars
  condlist = [r <= R, (r > R) & (r <= (R+d)), r > (R+d)]
Scripts/disk.py:55: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  return np.piecewise(r, condlist, funclist)


ValueError: setting an array element with a sequence.


IPython CPU timings (estimated):
  User   :     396.14 s.
  System :       0.00 s.
Wall time:     396.14 s.


In [ ]:
#Set remaining necessary variables
r0 = np.linspace(0, 10, 20)

#Define remaining necessary functions.
vtotal = lambda r: np.sqrt(vcdm(r)**2+vd**2+vbh(r)**2+vb**2)
vtotalv = np.vectorize(vtotal)

In [ ]:
#Plot

#Will not work until necessary code is determined and set up.

fig = plt.figure(figsize=(9.0,8.0))                #size of the plot
ax = plt.axes()
ax.grid(True)
ax.yaxis.grid(True,which='minor',linestyle='--')

plt.plot(r0, vtotalv(r0), linestyle='solid', label='Velocity')          

fig.suptitle('Total Rotation Curve NGC5533', fontsize=18)
ax.set(title='NGC 5533')              #labeling axes
ax.set_xlim(0,10)
ax.set_xlabel('r (kpc)', fontsize=14)
ax.set_ylabel('v (km/s)', fontsize=14)
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=14);

plt.savefig('Outputs/total.png')